# Monster Chess — Play Against the AI

Run **Setup** and **Helpers** once, then run **Play** for each new game. Type moves in the text box below the board and press Enter.

In [42]:
# === Setup (run once) ===
import os, sys, json, time, threading
import chess, chess.svg
import ipywidgets as widgets
from IPython.display import display, SVG, HTML

from config import MODEL_DIR, RAW_DATA_DIR
from monster_chess import MonsterChessGame
from mcts import MCTS

# --- Configuration ---
PLAY_AS = "black"         # "black" or "white"
USE_HEURISTIC = True      # True = heuristic eval, False = neural network
MODEL_PATH = os.path.join(MODEL_DIR, "best_value_net.pt")
SIMULATIONS = 800         # MCTS search depth (higher = stronger + slower)

# Load evaluator
eval_fn = None
if not USE_HEURISTIC and os.path.exists(MODEL_PATH):
    from evaluation import NNEvaluator
    eval_fn = NNEvaluator(MODEL_PATH)
    print(f"Model loaded: {MODEL_PATH} (CUDA: {eval_fn.device.type == 'cuda'})")
else:
    print("Using heuristic evaluation")

engine = MCTS(num_simulations=SIMULATIONS, eval_fn=eval_fn)
print(f"Playing as {PLAY_AS.upper()} | {SIMULATIONS} simulations")

Using heuristic evaluation
Playing as BLACK | 800 simulations


In [43]:
# === Helpers (run once) ===

def make_board_svg(board, last_move=None, flip=False, size=480):
    """Return SVG string for the board."""
    arrows = []
    if last_move:
        if isinstance(last_move, tuple):
            m1, m2 = last_move
            arrows.append(chess.svg.Arrow(m1.from_square, m1.to_square, color="#4a90d9aa"))
            if m2 != chess.Move.null():
                arrows.append(chess.svg.Arrow(m2.from_square, m2.to_square, color="#50c878aa"))
        else:
            arrows.append(chess.svg.Arrow(last_move.from_square, last_move.to_square, color="#4a90d9aa"))
    return chess.svg.board(board, arrows=arrows, flipped=flip, size=size, coordinates=True)


def make_eval_html(value):
    """Return HTML string for eval bar."""
    pct = max(2, min(98, int((value + 1) / 2 * 100)))
    if value > 0.3:
        label = f"<b>+{value:.2f}</b> White"
    elif value < -0.3:
        label = f"<b>{value:.2f}</b> Black"
    else:
        label = f"{value:.2f} Even"
    return f"""<div style="width:480px;height:22px;border:1px solid #888;border-radius:4px;overflow:hidden;display:flex;margin:4px 0">
      <div style="width:{pct}%;background:#eee"></div><div style="width:{100-pct}%;background:#333"></div>
    </div><div style="font-family:monospace;font-size:13px">{label}</div>"""


def parse_move(text, board):
    """Parse UCI or SAN into a chess.Move."""
    text = text.strip()
    try:
        move = chess.Move.from_uci(text)
        if move in board.pseudo_legal_moves or move in board.legal_moves:
            return move
    except (ValueError, chess.InvalidMoveError):
        pass
    try:
        return board.parse_san(text)
    except (ValueError, chess.InvalidMoveError, chess.AmbiguousMoveError):
        return None


def save_game(records, result):
    """Save game records to human_games directory."""
    for rec in records:
        rec["game_result"] = result
    out_dir = os.path.join(RAW_DATA_DIR, "human_games")
    os.makedirs(out_dir, exist_ok=True)
    existing = len([f for f in os.listdir(out_dir) if f.endswith(".jsonl")])
    path = os.path.join(out_dir, f"game_{existing:05d}.jsonl")
    with open(path, "w") as f:
        for rec in records:
            f.write(json.dumps(rec) + "\n")
    return path, len(records)


print("Helpers loaded.")

Helpers loaded.


## Play a Game

Run the cell below for each new game. Type moves in the text box (UCI like `e7e5` or SAN like `Nc6`) and press **Enter**. Type `quit` to resign.

In [44]:
# === PLAY A GAME (run for each new game) ===

class GameUI:
    """Widget-based Monster Chess game UI with inline focus management."""

    # Invisible img trick: onerror fires JS to focus the input widget.
    # This is the only reliable way to retain focus in VS Code notebooks.
    FOCUS_JS = (
        '<img src="x" style="display:none" onerror="'
        "setTimeout(()=>{let i=this.closest('.widget-vbox')"
        "?.querySelector('input.widget-input');"
        "if(i){i.focus();i.select();}},50)"
        '">'
    )

    def __init__(self):
        self.human_is_white = (PLAY_AS == "white")
        self.flip = self.human_is_white
        self.game = MonsterChessGame()
        self.records = []
        self.move_number = 0
        self.game_over = False
        self.awaiting_m2 = False
        self.pending_m1 = None
        self._focus_counter = 0  # forces HTML re-render each time

        # Widgets
        self.board_html = widgets.HTML()
        self.eval_html = widgets.HTML(value="")
        self.status = widgets.HTML(value=self._fmt("New game — AI is thinking..."))
        self.move_input = widgets.Text(
            placeholder="Type move here (e.g. e7e5, Nc6) then Enter",
            layout=widgets.Layout(width="480px"),
            continuous_update=False,
        )
        self.move_input.observe(self._on_value_change, names="value")
        self.log_html = widgets.HTML(value="")
        self.move_log = []
        # Hidden widget that fires focus JS
        self.focuser = widgets.HTML(value="")

        display(widgets.VBox([
            self.board_html,
            self.eval_html,
            self.status,
            self.move_input,
            self.focuser,
            self.log_html,
        ]))

        self._refresh_board()

        if not self.human_is_white:
            self._ai_turn()
        else:
            self.status.value = self._fmt("Your turn — enter move 1/2")
            self._refocus()

    def _fmt(self, text, color="#333"):
        return (
            f'<div style="font-family:monospace;font-size:14px;'
            f'color:{color};margin:4px 0;font-weight:bold">{text}</div>'
        )

    def _refocus(self):
        """Inject JS to pull focus back to the text input."""
        self._focus_counter += 1
        # The counter changes the HTML value, forcing a re-render which triggers onerror
        self.focuser.value = f'<!-- {self._focus_counter} -->{self.FOCUS_JS}'

    def _refresh_board(self, last_move=None):
        self.board_html.value = make_board_svg(
            self.game.board, last_move=last_move, flip=self.flip
        )

    def _update_log(self):
        rows = "".join(
            f"<tr><td style='padding:1px 8px;color:#666'>{i+1}.</td>"
            f"<td style='padding:1px 8px'>{m}</td></tr>"
            for i, m in enumerate(self.move_log)
        )
        self.log_html.value = (
            f'<div style="max-height:150px;overflow-y:auto;font-family:monospace;'
            f'font-size:13px;margin-top:4px"><table>{rows}</table></div>'
        )

    def _end_game(self, result, msg):
        self.game_over = True
        self.move_input.disabled = True
        self.status.value = self._fmt(msg, color="#c00" if result != 0 else "#888")
        if self.records:
            path, n = save_game(self.records, result)
            self.status.value += self._fmt(
                f"Saved {n} positions to {os.path.basename(path)}", color="#080"
            )

    def _on_value_change(self, change):
        """Fires when user presses Enter (continuous_update=False)."""
        if self.game_over:
            return
        text = change["new"].strip()
        if not text:
            return
        # Clear the input immediately
        self.move_input.unobserve(self._on_value_change, names="value")
        self.move_input.value = ""
        self.move_input.observe(self._on_value_change, names="value")

        if text.lower() in ("quit", "resign", "q"):
            result = -1 if self.human_is_white else 1
            winner = "Black" if self.human_is_white else "White"
            self._end_game(result, f"You resigned. {winner} wins!")
            return

        if self.awaiting_m2:
            self._handle_white_m2(text)
        elif self.human_is_white:
            self._handle_white_m1(text)
        else:
            self._handle_black_move(text)

    def _handle_black_move(self, text):
        self.game.board.turn = chess.BLACK
        legal = list(self.game.board.legal_moves)
        if not legal:
            self._end_game(0, "No legal moves — stalemate!")
            return
        move = parse_move(text, self.game.board)
        if move is None or move not in legal:
            hints = ", ".join(m.uci() for m in legal[:12])
            more = f"... +{len(legal)-12}" if len(legal) > 12 else ""
            self.status.value = self._fmt(f"Invalid. Legal: {hints}{more}", "#c00")
            self._refocus()
            return

        self.game.apply_action(move)
        self.move_log.append(f"You: {move.uci()}")
        self._update_log()
        self._refresh_board(last_move=move)
        self.move_number += 1

        if self.game.is_terminal():
            r = self.game.get_result()
            self._end_game(r, "White wins!" if r>0 else "Black wins!" if r<0 else "Draw")
            return

        self._ai_turn()

    def _handle_white_m1(self, text):
        self.game.board.turn = chess.WHITE
        legal = list(self.game.board.pseudo_legal_moves)
        m1 = parse_move(text, self.game.board)
        if m1 is None or m1 not in legal:
            hints = ", ".join(m.uci() for m in legal[:12])
            self.status.value = self._fmt(f"Invalid move 1. Try: {hints}...", "#c00")
            self._refocus()
            return

        self.game.board.push(m1)
        if self.game.board.king(chess.BLACK) is None:
            self.game.board.pop()
            action = (m1, chess.Move.null())
            self.game.apply_action(action)
            self.move_log.append(f"You: {m1.uci()} (king captured!)")
            self._update_log()
            self._refresh_board(last_move=action)
            self.move_number += 1
            if self.game.is_terminal():
                self._end_game(self.game.get_result(), "White wins!")
            return

        self._refresh_board(last_move=m1)
        self.pending_m1 = m1
        self.awaiting_m2 = True
        self.game.board.pop()
        self.status.value = self._fmt(f"Move 1: {m1.uci()} — now enter move 2/2")
        self.move_input.placeholder = "Move 2/2..."
        self._refocus()

    def _handle_white_m2(self, text):
        m1 = self.pending_m1
        self.game.board.push(m1)
        self.game.board.turn = chess.WHITE
        legal2 = list(self.game.board.pseudo_legal_moves)
        m2 = parse_move(text, self.game.board)
        if m2 is None or m2 not in legal2:
            hints = ", ".join(m.uci() for m in legal2[:12])
            self.status.value = self._fmt(f"Invalid move 2. Try: {hints}...", "#c00")
            self.game.board.turn = chess.BLACK
            self.game.board.pop()
            self._refocus()
            return

        self.game.board.turn = chess.BLACK
        self.game.board.pop()
        action = (m1, m2)
        self.awaiting_m2 = False
        self.pending_m1 = None
        self.move_input.placeholder = "Type move here (e.g. e2e4)"

        self.game.apply_action(action)
        self.move_log.append(f"You: {m1.uci()} + {m2.uci()}")
        self._update_log()
        self._refresh_board(last_move=action)
        self.status.value = self._fmt(f"You played: {m1.uci()} + {m2.uci()}")
        self.move_number += 1

        if self.game.is_terminal():
            r = self.game.get_result()
            self._end_game(r, "White wins!" if r>0 else "Black wins!" if r<0 else "Draw")
            return

        self._ai_turn()

    def _ai_turn(self):
        is_white = self.game.is_white_turn
        side = "White" if is_white else "Black"
        self.status.value = self._fmt(f"AI ({side}) thinking...", "#888")
        self.move_input.disabled = True

        t0 = time.time()
        action, action_probs, root_value = engine.get_best_action(
            self.game, temperature=0.1
        )
        elapsed = time.time() - t0

        if action is None:
            self._end_game(0, "AI has no legal moves — stalemate!")
            return

        self.records.append({
            "fen": self.game.fen(),
            "mcts_value": round(root_value, 4),
            "policy": action_probs,
            "current_player": "white" if is_white else "black",
        })

        self.game.apply_action(action)
        self.move_number += 1

        if is_white:
            m1, m2 = action
            move_str = f"{m1.uci()} + {m2.uci()}"
        else:
            move_str = action.uci()

        self.move_log.append(f"AI ({side}): {move_str}  [{elapsed:.1f}s]")
        self._update_log()
        self._refresh_board(last_move=action)
        self.eval_html.value = make_eval_html(root_value)
        self.status.value = self._fmt(
            f"AI: {move_str}  ({elapsed:.1f}s, eval {root_value:+.2f}) — your turn"
        )
        self.move_input.disabled = False

        if self.game.is_terminal():
            r = self.game.get_result()
            self._end_game(r, "White wins!" if r>0 else "Black wins!" if r<0 else "Draw")
            return

        # Pull focus back to input after AI finishes
        self._refocus()


# Start a new game
ui = GameUI()